# Goal

<h3 style="color:blue">assess the quality of summaries written by students</h3>
<h3 style="color:indigo">evaluate how well a student represents the main idea and details of a source text, as well as the clarity, precision, and fluency of the language used in the summary</h3>
<h3 style="color:red">Freely & publicly available external data is <b>allowed</b>, including pre-trained models</h3>
<h3>This is Multi-Output problem</h3>

### Use Hugging Face Library
### Use NLTK
### Use Tensorflow

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import math
import subprocess
from tqdm import tqdm
import pickle

In [3]:
import tensorflow as tf

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score, median_absolute_error

In [5]:
import transformers
from transformers import AutoTokenizer, TFBertModel

In [6]:
prompts_train = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')
summaries_train = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
prompts_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')
summaries_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')

In [7]:
train = pd.merge(prompts_train, summaries_train, on='prompt_id')
test = pd.merge(prompts_test, summaries_test, on='prompt_id')

In [8]:
train.rename(columns = {'text' : 'summary'}, inplace=True)
test.rename(columns = {'text' : 'summary'}, inplace=True)

In [9]:
train.head(2)

,prompt_id,prompt_question,prompt_title,prompt_text,student_id,summary,content,wording
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00791789cc1f,1 element of an ideal tragedy is that it shoul...,-0.210614,-0.471415
1,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,0086ef22de8f,The three elements of an ideal tragedy are: H...,-0.970237,-0.417058


In [10]:
train['summary'][0]

'1 element of an ideal tragedy is that it should be arranged on a complex plan.  Another element of an ideal tragedy is that it should only have one main issue. The last element of an ideal tragedy is that it should have a double thread plot and an opposite catastrophe for both good and bad.'

In [11]:
columns_needed = ["prompt_text", "summary"]

In [12]:
train_data = train[columns_needed]
test_data = test[columns_needed]

In [13]:
#from transformers import XLNetTokenizer, TFXLNetModel
#tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
#model = TFXLNetModel.from_pretrained('xlnet-base-cased', return_dict=True)

#from transformers import RobertaTokenizer, TFRobertaModel
#tokenizer = RobertaTokenizer.from_pretrained('roberta-base-cased')
#model = TFRobertaModel.from_pretrained('roberta-base-cased', return_dict=True)

from transformers import AutoTokenizer, TFBertModel
model = TFBertModel.from_pretrained('/kaggle/input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased')

Some layers from the model checkpoint at /kaggle/input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at /kaggle/input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


### Next time use prepare_tf_dataset which is used to directly tokenize and data colat and
### make dataset compatible with tensorflow
####       https://huggingface.co/docs/transformers/v4.31.0/en/main_classes/model#transformers.TFPreTrainedModel.prepare_tf_dataset

In [14]:

def vectorize_dataframe(dataframe, col):
    vectors = []
    for text in tqdm(dataframe[col].tolist()):
        text_tokens = tokenizer(text, return_tensors="tf",max_length = 4000, padding='max_length', truncation=True)

        vectors.append(text_tokens['input_ids'])
    return vectors
    

In [15]:
train_data['prompt_text_embedded'] = vectorize_dataframe(train_data, 'prompt_text')
train_data['summary_embedded'] = vectorize_dataframe(train_data, 'summary')

100%|██████████| 7165/7165 [00:12<00:00, 553.95it/s]


In [16]:
test_data['prompt_text_embedded'] = vectorize_dataframe(test_data, 'prompt_text')
test_data['summary_embedded'] = vectorize_dataframe(test_data, 'summary')

100%|██████████| 4/4 [00:00<00:00, 646.35it/s]


In [17]:
traning_set = train_data[['prompt_text_embedded', 'summary_embedded']]
testing_set = test_data[['prompt_text_embedded', 'summary_embedded']]

### Take average of embeddings  [Not required, just checking]

In [18]:
target1 = np.array(train['content'])
target1 = target1.astype('float32')

target2 = np.array(train['wording'])
target2 = target2.astype('float32')

#target = (target1, target2)

In [19]:
def convert_tensor_to_numpy(tensor):
        return np.array(tensor, dtype='int64')

traning_set = traning_set.applymap(convert_tensor_to_numpy)
testing_set = testing_set.applymap(convert_tensor_to_numpy)

In [20]:
def prepare_dataset(dataset):
    # Flatten the nested arrays in the DataFrame
    dataset['prompt_text_embedded'] = dataset['prompt_text_embedded'].apply(lambda x: x.flatten())
    dataset['summary_embedded'] = dataset['summary_embedded'].apply(lambda x: x.flatten())
    
    feature1 = np.array(dataset['prompt_text_embedded'].tolist())
    feature2 = np.array(dataset['summary_embedded'].tolist())
    
    features = np.concatenate((feature1, feature2), axis=1)
    
    return features

In [21]:
features = prepare_dataset(traning_set)

In [22]:
features_for_test = prepare_dataset(testing_set)

In [23]:
#X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [24]:
from tensorflow.keras.layers import Dense, Input, Flatten


In [25]:
# Define the model architecture
model_content = tf.keras.Sequential([
      tf.keras.layers.Dense(256, activation='linear'),
      tf.keras.layers.Dense(128, activation='linear'),
      tf.keras.layers.Dense(64, activation='linear'),
      tf.keras.layers.Dense(32, activation='linear'),
      tf.keras.layers.Dense(1, activation='linear')
])

model_wording = tf.keras.Sequential([
      tf.keras.layers.Dense(256, activation='linear'),
      tf.keras.layers.Dense(128, activation='linear'),
      tf.keras.layers.Dense(64, activation='linear'),
      tf.keras.layers.Dense(32, activation='linear'),
      tf.keras.layers.Dense(1, activation='linear')
])

In [26]:
model_content.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])
model_wording.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

In [37]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])


In [40]:
model.fit(features, target1, epochs=1)

ResourceExhaustedError: Graph execution error:

Detected at node 'tf_bert_model/bert/encoder/layer_._0/attention/self/MatMul' defined at (most recent call last):
    File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 728, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
      s

In [27]:
# Traiactivity_regularizern your model using model.fit()
history1 = model_content.fit(features, target1, epochs=56,batch_size=50, validation_split=0.2)

Epoch 1/56
115/115 [==============================] - 4s 8ms/step - loss: 99325488.0000 - mae: 3939.4507 - mse: 99325488.0000 - val_loss: 51554.1953 - val_mae: 165.3712 - val_mse: 51554.1953
Epoch 2/56
115/115 [==============================] - 1s 6ms/step - loss: 170629.9531 - mae: 142.7761 - mse: 170629.9531 - val_loss: 65312.3867 - val_mae: 139.9648 - val_mse: 65312.3867
Epoch 3/56
115/115 [==============================] - 1s 6ms/step - loss: 70863.6953 - mae: 114.6958 - mse: 70863.6953 - val_loss: 78860.1562 - val_mae: 129.1164 - val_mse: 78860.1562
Epoch 4/56
115/115 [==============================] - 1s 7ms/step - loss: 43439.0078 - mae: 95.3637 - mse: 43439.0078 - val_loss: 20185.8359 - val_mae: 113.2654 - val_mse: 20185.8359
Epoch 5/56
115/115 [==============================] - 1s 6ms/step - loss: 31702.3945 - mae: 80.5353 - mse: 31702.3945 - val_loss: 21234.1113 - val_mae: 90.4481 - val_mse: 21234.1094
Epoch 6/56
115/115 [==============================] - 1s 6ms/step - loss: 

In [28]:
history2 = model_wording.fit(features, target2, epochs=56,batch_size=50, validation_split=0.2)

Epoch 1/56
115/115 [==============================] - 3s 8ms/step - loss: 125816368.0000 - mae: 4409.4570 - mse: 125816368.0000 - val_loss: 89501.3516 - val_mae: 218.0971 - val_mse: 89501.3516
Epoch 2/56
115/115 [==============================] - 1s 6ms/step - loss: 150924.6562 - mae: 180.6431 - mse: 150924.6562 - val_loss: 103685.1484 - val_mae: 198.6741 - val_mse: 103685.1484
Epoch 3/56
115/115 [==============================] - 1s 6ms/step - loss: 259850.5625 - mae: 165.5742 - mse: 259850.5625 - val_loss: 168483.4219 - val_mae: 180.4954 - val_mse: 168483.4219
Epoch 4/56
115/115 [==============================] - 1s 6ms/step - loss: 141047.4531 - mae: 137.4574 - mse: 141047.4531 - val_loss: 43938.7539 - val_mae: 123.9949 - val_mse: 43938.7539
Epoch 5/56
115/115 [==============================] - 1s 7ms/step - loss: 97033.9844 - mae: 110.6730 - mse: 97033.9844 - val_loss: 45522.3828 - val_mae: 112.5610 - val_mse: 45522.3828
Epoch 6/56
115/115 [==============================] - 1s 6ms/

In [29]:
evaluate_on_train_content = model_content.evaluate(features, target1)
evaluate_on_train_wording = model_wording.evaluate(features, target2)

224/224 [==============================] - 1s 4ms/step - loss: 1003.4942 - mae: 12.3596 - mse: 1003.4942


In [30]:
print('evaluate_on_train_content', evaluate_on_train_content)
print('evaluate_on_train_wording',evaluate_on_train_wording)

evaluate_on_train_content [23611.341796875, 68.01499938964844, 23611.341796875]
evaluate_on_train_wording [1003.4942016601562, 12.359598159790039, 1003.4942016601562]


In [31]:
content_prediction = model_content.predict(features)
wording_prediction = model_wording.predict(features)

224/224 [==============================] - 1s 2ms/step


### Predict on test

In [32]:
test_pred_content = model_content.predict(features_for_test)
test_pred_wording = model_wording.predict(features_for_test)

1/1 [==============================] - 0s 19ms/step


## submission

In [33]:
test_pred_content = test_pred_content.reshape(-1)
test_pred_wording = test_pred_wording.reshape(-1)

In [34]:
submission = pd.DataFrame({
    'student_id' : test['student_id'],
    'content' : test_pred_content,
    'wording' : test_pred_wording
})

In [35]:
submission.to_csv('submission.csv', index=False)

In [36]:
submission.head()

,student_id,content,wording
0,000000ffffff,7.110975,-2.385606
1,222222cccccc,7.110075,-2.385587
2,111111eeeeee,7.110529,-2.385564
3,333333dddddd,7.109636,-2.385571
